# European Soccer

## Extracting Data
dataset available at: https://www.kaggle.com/hugomathien/soccer/data

In [85]:
import csv
import sqlite3 as lite

In [1]:
import pandas as pd
import numpy as np

In [86]:
con = lite.connect('database.sqlite')

team_df = pd.read_sql_query("SELECT * from Team", con)

In [88]:
team_df.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [93]:
team_names = team_df.set_index('team_api_id')['team_long_name'].to_dict()

## Ligue 1 conforama

In [2]:
df = pd.read_csv('ligue1_conforama.csv')

In [3]:
df.head()

,id,id.1,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,4769,4769,4769,4769,2008/2009,1,2008-08-09 00:00:00,483129,8583,9830,...,3.8,2.0,2.9,3.75,2.10,3.00,3.75,2.00,2.88,3.75
1,4770,4770,4769,4769,2008/2009,1,2008-08-09 00:00:00,483130,9827,7819,...,7.5,1.6,3.3,5.50,1.57,3.60,6.00,1.50,3.50,6.50
2,4771,4771,4769,4769,2008/2009,1,2008-08-09 00:00:00,483131,9746,9831,...,3.1,2.3,2.9,3.00,2.35,2.90,3.20,2.38,2.88,2.88
3,4772,4772,4769,4769,2008/2009,1,2008-08-09 00:00:00,483132,8682,8689,...,4.0,2.0,2.9,3.75,2.00,3.10,3.75,1.91,3.00,4.00
4,4773,4773,4769,4769,2008/2009,1,2008-08-10 00:00:00,483133,9748,9941,...,12.0,1.3,4.0,10.00,1.36,4.25,9.00,1.30,4.33,9.00


### On mesure la probabilité empirique pour l'équipe qui accueille de V/N/D

In [4]:
df['goal_diff'] = df['home_team_goal'] - df['away_team_goal']

In [5]:
def delta_to_result(delta):
    if delta > 0:
        return 'V'
    if delta == 0:
        return 'N'
    else:
        return 'D'

In [6]:
df['home_team_result'] = df['goal_diff'].map(delta_to_result)

In [7]:
df['home_team_result'].head()

0    V
1    V
2    V
3    D
4    V
Name: home_team_result, dtype: object

In [9]:
df.groupby('home_team_result').count()

,id,id.1,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,...,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA,goal_diff
home_team_result,,,,,,,,,,,,,,,,,,,,,
D,822,822,822,822,822,822,822,822,822,822,...,822,822,822,495,495,495,495,495,495,822
N,859,859,859,859,859,859,859,859,859,859,...,859,859,859,555,555,555,555,555,555,859
V,1359,1359,1359,1359,1359,1359,1359,1359,1359,1359,...,1356,1356,1356,847,847,847,847,847,847,1359


In [10]:
pV = 1359. / len(df.index)
pN = 859. / len(df.index)
pD = 822. / len(df.index)

In [11]:
# empiric probas of V/N/D for the home_team
p = np.array([pV, pN, pD])
p

array([ 0.44703947,  0.28256579,  0.27039474])

## On aggrège les résultats de la première saison

In [30]:
prev_season = df[df['season'] == '2008/2009']

In [32]:
home_avg = prev_season[['home_team_api_id', 'home_team_goal']].groupby('home_team_api_id').mean().reset_index()

In [33]:
away_avg = prev_season[['away_team_api_id', 'away_team_goal']].groupby('away_team_api_id').mean().reset_index()

In [34]:
prev_teams = prev_season['home_team_api_id'].drop_duplicates().tolist()

In [50]:
home_avg.head()

,home_team_api_id,home_team_goal
0,7819,1.157895
1,8481,1.000000
2,8583,0.947368
3,8592,1.842105
4,8639,1.789474


In [58]:
sum(home_avg[home_avg['home_team_api_id']==7819]['home_team_goal'])

1.1578947368421053

### On construit le graph

In [22]:
import networkx as nx

In [23]:
season = df[df['season'] == '2009/2010']

In [70]:
teams = season['home_team_api_id'].drop_duplicates().tolist()

In [124]:
g = nx.DiGraph()

In [125]:
for team_id in teams:
    g.add_node((team_id, 'h'), name = team_names[team_id], ha = 'h')
    g.add_node((team_id, 'a'), name = team_names[team_id], ha = 'a')

In [126]:
for i in teams:
    
    if i in prev_teams:
        for j in teams:
            if i != j:
                wh = sum(home_avg[home_avg['home_team_api_id']==i]['home_team_goal'])
                g.add_edge((i, 'h'), (j, 'a'), weight = wh)

                wa = sum(away_avg[away_avg['away_team_api_id']==i]['away_team_goal'])
                g.add_edge((i, 'a'), (j, 'h'), weight = wa)
            
    else:
        for j in teams:
            if i != j:
                wh = min(home_avg['home_team_goal'])
                g.add_edge((i, 'h'), (j, 'a'), weight = wh)

                wa = min(away_avg['away_team_goal'])
                g.add_edge((i, 'a'), (j, 'h'), weight = wa)

In [99]:
nx.write_gexf(g, "L1.gexf")

In [81]:
pr = nx.pagerank(g, alpha=1.)

In [82]:
sorted([(key, value) for key, value in pr.items()], key = lambda x: x[1])

[((9829, 'a'), 0.025000000000000005),
 ((9873, 'a'), 0.025000000000000005),
 ((9831, 'a'), 0.025000000000000005),
 ((9855, 'a'), 0.025000000000000005),
 ((10249, 'a'), 0.025000000000000005),
 ((9827, 'a'), 0.025000000000000005),
 ((9748, 'a'), 0.025000000000000005),
 ((8481, 'a'), 0.025000000000000005),
 ((9874, 'a'), 0.025000000000000005),
 ((9851, 'a'), 0.025000000000000005),
 ((4170, 'a'), 0.025000000000000005),
 ((8689, 'a'), 0.025000000000000005),
 ((8583, 'a'), 0.025000000000000005),
 ((8639, 'a'), 0.025000000000000005),
 ((9847, 'a'), 0.025000000000000005),
 ((8592, 'a'), 0.025000000000000005),
 ((9941, 'a'), 0.025000000000000005),
 ((8588, 'a'), 0.025000000000000005),
 ((8682, 'a'), 0.025000000000000005),
 ((9853, 'a'), 0.02500000000000001),
 ((8682, 'h'), 0.025000000000000012),
 ((8588, 'h'), 0.025000000000000012),
 ((9829, 'h'), 0.025000000000000012),
 ((8583, 'h'), 0.025000000000000012),
 ((9851, 'h'), 0.025000000000000012),
 ((9748, 'h'), 0.025000000000000012),
 ((8481, 'h'

In [127]:
for stage in range(1,39):
    for (u,v) in g.edges():
        g[u][v]['weight'] *= 0.5
        
    day = season[season['stage']==stage]
    
    for row in day[['home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal']].iterrows():
        row = row[1]
        ht_id = row['home_team_api_id']
        at_id = row['away_team_api_id']
        htg = row['home_team_goal']
        atg = row['away_team_goal']

        g[(ht_id, 'h')][(at_id, 'a')]['weight'] += 0.5 * htg
        g[(at_id, 'a')][(ht_id, 'h')]['weight'] += 0.5 * atg

In [133]:
pr = nx.pagerank(g, alpha=0.7)

In [135]:
spr = sorted([((team_names[key[0]], key[1]), value) for key, value in pr.items()], key = lambda x: x[1])

In [138]:
scores = {}
for ((team, ha), pr) in spr:
    if team in scores:
        scores[team] += pr
    else:
        scores[team] = pr

In [140]:
sorted([(key, value) for key, value in scores.items()], key = lambda x: x[1]) 

[('Toulouse FC', 0.023625509736124959),
 ("US Boulogne Cote D'Opale", 0.029820104575955624),
 ('Olympique Lyonnais', 0.033323692289347499),
 ('Le Mans FC', 0.037792703007271014),
 ('AS Monaco', 0.038609723257016434),
 ('Olympique de Marseille', 0.038934731142824835),
 ('Grenoble Foot 38', 0.040753128341530886),
 ('Paris Saint-Germain', 0.046458384406384889),
 ('LOSC Lille', 0.047694811892405653),
 ('Valenciennes FC', 0.051481061921394886),
 ('Stade Rennais FC', 0.051612293400432586),
 ('FC Sochaux-Montbéliard', 0.05353634904798827),
 ('AS Saint-Étienne', 0.054220800109112781),
 ('AJ Auxerre', 0.055611240408380447),
 ('OGC Nice', 0.056039812085183036),
 ('Girondins de Bordeaux', 0.057462729615491905),
 ('Montpellier Hérault SC', 0.060830299379942131),
 ('AS Nancy-Lorraine', 0.062619163627455329),
 ('FC Lorient', 0.074040343231732475),
 ('RC Lens', 0.085533118524024496)]